In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.svm import SVC  
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import FastICA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import decomposition
from sklearn.feature_selection import SelectPercentile as sp
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFECV

In [2]:
df = pd.read_csv('Linux_process_2.csv')

In [3]:
df.head()

,ts,PID,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,CMD,label,type
0,1556739928,2579,0,1,0,norm,0,120,0,0,-,0,S,0.0,upstart-dbus-b,0,normal
1,1556739933,3739,0,1,0,norm,0,120,0,0,-,0,S,0.0,kworker/0:1,0,normal
2,1556739938,3752,0,4,0,norm,0,120,0,1,-,0,S,0.0,gnome-terminal,0,normal
3,1556739943,2596,0,4,0,norm,0,120,0,3,-,0,S,0.0,unity-settings,0,normal
4,1556739948,1400,0,1,0,norm,0,120,0,2,-,0,S,0.0,irqbalance,0,normal


In [4]:
df = df.drop(['ts', 'PID', 'CMD'], axis = 1)
df

,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,label,type
0,0,1,0,norm,0,120,0,0,-,0,S,0.0,0,normal
1,0,1,0,norm,0,120,0,0,-,0,S,0.0,0,normal
2,0,4,0,norm,0,120,0,1,-,0,S,0.0,0,normal
3,0,4,0,norm,0,120,0,3,-,0,S,0.0,0,normal
4,0,1,0,norm,0,120,0,2,-,0,S,0.0,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927963,0,0,0,0,0,0,0,0,-,0,E,0.0,0,normal
927964,0,0,0,0,0,0,0,0,-,0,E,0.0,0,normal
927965,0,0,0,0,0,0,0,0,-,0,E,0.0,0,normal
927966,0,0,0,0,0,0,0,0,-,0,E,0.0,0,normal


In [5]:
df.isnull().sum()

TRUN      0
TSLPI     0
TSLPU     0
POLI      0
NICE      0
PRI       0
RTPR      0
CPUNR     0
Status    0
EXC       0
State     0
CPU       0
label     0
type      0
dtype: int64

In [6]:
target = df.loc[:, ('type')]

In [7]:
df = df.drop('type', axis = 1)
df

,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,label
0,0,1,0,norm,0,120,0,0,-,0,S,0.0,0
1,0,1,0,norm,0,120,0,0,-,0,S,0.0,0
2,0,4,0,norm,0,120,0,1,-,0,S,0.0,0
3,0,4,0,norm,0,120,0,3,-,0,S,0.0,0
4,0,1,0,norm,0,120,0,2,-,0,S,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
927963,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927964,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927965,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927966,0,0,0,0,0,0,0,0,-,0,E,0.0,0


In [8]:
cat = []
for i in df.columns:
    if len(df[i].unique()) <= 328: cat.append(i)
df_cat = df[cat]
df_cat

,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,label
0,0,1,0,norm,0,120,0,0,-,0,S,0.0,0
1,0,1,0,norm,0,120,0,0,-,0,S,0.0,0
2,0,4,0,norm,0,120,0,1,-,0,S,0.0,0
3,0,4,0,norm,0,120,0,3,-,0,S,0.0,0
4,0,1,0,norm,0,120,0,2,-,0,S,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
927963,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927964,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927965,0,0,0,0,0,0,0,0,-,0,E,0.0,0
927966,0,0,0,0,0,0,0,0,-,0,E,0.0,0


In [9]:
label_encoder = preprocessing.LabelEncoder()
df['Status'] = label_encoder.fit_transform(df['Status'])
df['POLI'] = label_encoder.fit_transform(df['POLI'])
df['State'] = label_encoder.fit_transform(df['State'])

In [10]:
df

,TRUN,TSLPI,TSLPU,POLI,NICE,PRI,RTPR,CPUNR,Status,EXC,State,CPU,label
0,0,1,0,2,0,120,0,0,0,0,4,0.0,0
1,0,1,0,2,0,120,0,0,0,0,4,0.0,0
2,0,4,0,2,0,120,0,1,0,0,4,0.0,0
3,0,4,0,2,0,120,0,3,0,0,4,0.0,0
4,0,1,0,2,0,120,0,2,0,0,4,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
927963,0,0,0,1,0,0,0,0,0,0,1,0.0,0
927964,0,0,0,1,0,0,0,0,0,0,1,0.0,0
927965,0,0,0,1,0,0,0,0,0,0,1,0.0,0
927966,0,0,0,1,0,0,0,0,0,0,1,0.0,0


In [11]:
def lr(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_lr = LogisticRegression()
    model_lr.fit(x_train, y_train)
    preds = model_lr.predict(x_test)
    return accuracy_score(y_test, preds)

In [12]:
def dtc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_dtc = DecisionTreeClassifier()   
    model_dtc.fit(x_train, y_train)
    preds = model_dtc.predict(x_test)
    return accuracy_score(y_test, preds)

In [13]:
def rfc(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_rfc = RandomForestClassifier()    
    model_rfc.fit(x_train, y_train)
    preds = model_rfc.predict(x_test)
    return accuracy_score(y_test, preds)

In [14]:
def knn(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_knn = KNeighborsClassifier()
    model_knn.fit(x_train, y_train)
    preds = model_knn.predict(x_test)
    return accuracy_score(y_test, preds)

In [15]:
def svm(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_svm = SVC(kernel = 'sigmoid')
    model_svm.fit(x_train, y_train) 
    preds = model_svm.predict(x_test)
    return accuracy_score(y_test, preds)

In [16]:
def nb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_nb = GaussianNB()
    model_nb.fit(x_train, y_train)
    preds = model_nb.predict(x_test)
    return accuracy_score(y_test, preds)

In [17]:
def xgbb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_xgb = xgb.XGBClassifier()
    model_xgb.fit(x_train, y_train)
    preds = model_xgb.predict(x_test)
    return accuracy_score(y_test, preds)

In [18]:
def adb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_adb = AdaBoostClassifier()
    model_adb.fit(x_train, y_train)
    preds = model_adb.predict(x_test)
    return accuracy_score(y_test, preds)

In [19]:
def mlp(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = MLPClassifier()    
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds)

In [20]:
def gb(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 43)
    model_mlp = GradientBoostingClassifier() 
    model_mlp.fit(x_train, y_train)
    preds = model_mlp.predict(x_test)
    return accuracy_score(y_test, preds)

In [21]:
target = label_encoder.fit_transform(target)

In [22]:
x = df
y = target

In [23]:
x.columns = x.columns.astype('str')

In [24]:
#without feature selection


print('lr - ', lr(x, y))
print('dtc - ', dtc(x, y))
print('rfc - ', rfc(x, y))
print('knn - ', knn(x, y))
# print('svm - ', svm(x, y))
print('Naive Bayes - ', nb(x, y))
print('XGB - ', xgbb(x, y))
print('ADB - ', adb(x, y))
print('MLP - ', mlp(x, y))
print('GB - ', gb(x, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  0.8729017820259994
dtc -  0.9266607038302244
rfc -  0.9266607038302244
knn -  0.9207912612117489
Naive Bayes -  0.8598589753260701
XGB -  0.9267648738644568
ADB -  0.8759514495799074
MLP -  0.9170662844704032
GB -  0.9226016645653057


In [24]:
def ica(x, i):
    ica = FastICA(n_components = i, random_state = 42)
    data_ica = ica.fit_transform(x)
    data_ica = pd.DataFrame(data_ica)
    return data_ica

In [25]:
def pca(x, i):
    pca = decomposition.PCA(n_components = i, random_state=42)
    data_pca = pca.fit_transform(x)
    data_pca = pd.DataFrame(data_pca)
    return data_pca

In [26]:
def mi(x, y):
    mi = sp(percentile = 50)
    data_mi = mi.fit(x, y)
    data_mi = mi.transform(x)
    data_mi = pd.DataFrame(data_mi)
    return data_mi

In [27]:
def smf(x, y):
    lasso_model = Lasso(alpha = 0.1)
    selector = SelectFromModel(lasso_model)
    X_selected = selector.fit_transform(x, y)
    selected_features_indices = selector.get_support(indices=True)
    selected_features_names = x.columns[selected_features_indices]
    return x[selected_features_names]

In [28]:
def sfs(x, y, model):
    sfs_selector = SequentialFeatureSelector(model, forward = True, k_features = len(x.columns) // 2)
    X_selected = sfs_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[sfs_selector.k_feature_idx_])
    return x[selected_features_names]

In [29]:
def rfe(x, y, model):
    rfecv_selector = RFECV(estimator = model, step = 1, cv = 5, scoring = 'accuracy')
    X_selected = rfecv_selector.fit_transform(x, y)
    selected_features_names = list(x.columns[rfecv_selector.support_])
    return x[selected_features_names]

In [ ]:
def cfs(x):
    corr_matrix = x.corr()
    high_corr_features = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > 0.8:
                colname = corr_matrix.columns[i]
                high_corr_features.append(colname)

    return x.drop(high_corr_features, axis=1)

In [ ]:
def fir(x, y):
    rfc = RandomForestClassifier(n_estimators=100)
    rfc.fit(x, y)
    feature_importances = list(zip(x.columns, rfc.feature_importances_))
    a = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    ls = []
    for i in a[:len(a)//2 + 1]: ls.append(i[0])
    return x[ls]

In [30]:
data_ica = ica(x, len(x.columns) // 2)

C:\Users\serev\anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [31]:
data_pca = pca(x, len(x.columns) // 2)

In [32]:
data_mi = mi(x, y)

C:\Users\serev\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 6 12] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\serev\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
C:\Users\serev\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [33]:
data_smf = smf(x, y)

In [ ]:
data_cfs = cfs(x)

In [ ]:
data_fir = fir(x, y)

In [35]:
#with feature selection ica


print('lr - ', lr(data_ica, y))
print('dtc - ', dtc(data_ica, y))
print('rfc - ', rfc(data_ica, y))
print('knn - ', knn(data_ica, y))
# print('svm - ', svm(data_ica, y))
print('Naive Bayes - ', nb(data_ica, y))
print('XGB - ', xgbb(data_ica, y))
print('ADB - ', adb(data_ica, y))
print('MLP - ', mlp(data_ica, y))
print('GB - ', gb(data_ica, y))

lr -  0.816588898348007
dtc -  0.9180253671993707
rfc -  0.9180505116903923
knn -  0.9185102966690734
Naive Bayes -  0.09638961029630987
XGB -  0.9265313893049704
ADB -  0.683017769971012
MLP -  0.8165781221375691
GB -  0.90163475112342


In [36]:
#with feature selection pca


print('lr - ', lr(data_pca, y))
print('dtc - ', dtc(data_pca, y))
print('rfc - ', rfc(data_pca, y))
print('knn - ', knn(data_pca, y))
# print('svm - ', svm(data_pca, y))
print('Naive Bayes - ', nb(data_pca, y))
print('XGB - ', xgbb(data_pca, y))
print('ADB - ', adb(data_pca, y))
print('MLP - ', mlp(data_pca, y))
print('GB - ', gb(data_pca, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  0.816452399682461
dtc -  0.926488284463219
rfc -  0.9265385734452622
knn -  0.9196777194665058
Naive Bayes -  0.07261369800029455
XGB -  0.926621191058619
ADB -  0.7911031606625214
MLP -  0.9060637736133711
GB -  0.9181762341455004


In [34]:
#with feature selection mi


print('lr - ', lr(data_mi, y))
print('dtc - ', dtc(data_mi, y))
print('rfc - ', rfc(data_mi, y))
print('knn - ', knn(data_mi, y))
# print('svm - ', svm(data_mi, y))
print('Naive Bayes - ', nb(data_mi, y))
print('XGB - ', xgbb(data_mi, y))
print('ADB - ', adb(data_mi, y))
print('MLP - ', mlp(data_mi, y))
print('GB - ', gb(data_mi, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  0.8896551971866906
dtc -  0.9156222722717329
rfc -  0.915618680201587
knn -  0.9080537804742251
Naive Bayes -  0.8846298910525124
XGB -  0.9156258643418789
ADB -  0.8784227938403181
MLP -  0.9115883774978357
GB -  0.9138585658300735


In [35]:
#with feature selection smf


print('lr - ', lr(data_smf, y))
print('dtc - ', dtc(data_smf, y))
print('rfc - ', rfc(data_smf, y))
print('knn - ', knn(data_smf, y))
# print('svm - ', svm(data_smf, y))
print('Naive Bayes - ', nb(data_smf, y))
print('XGB - ', xgbb(data_smf, y))
print('ADB - ', adb(data_smf, y))
print('MLP - ', mlp(data_smf, y))
print('GB - ', gb(data_smf, y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr -  0.88731675952168
dtc -  0.8949606847922527
rfc -  0.8949499085818148
knn -  0.8898527610447177
Naive Bayes -  0.8800787381775992
XGB -  0.8949499085818148
ADB -  0.8845364972287179
MLP -  0.8918427679055717
GB -  0.8949427244415229


In [36]:
model_lr = LogisticRegression()
model_dtc = DecisionTreeClassifier()
model_rfc = RandomForestClassifier()
model_knn = KNeighborsClassifier()
model_svc = SVC(kernel = 'sigmoid')
model_nb = GaussianNB()
model_xgb = xgb.XGBClassifier()
model_adb = AdaBoostClassifier()
model_mlp = MLPClassifier()
model_gb = GradientBoostingClassifier()

In [38]:
#with feature selection sfs


print('lr - ', lr(sfs(x, y, model_lr), y))
print('dtc - ', dtc(sfs(x, y, model_dtc), y))
print('rfc - ', rfc(sfs(x, y, model_rfc), y))
print('knn - ', knn(sfs(x, y, model_knn), y))
# print('svm - ', svm(sfs(x, y, model_svc), y))
print('Naive Bayes - ', nb(sfs(x, y, model_nb), y))
print('XGB - ', xgbb(sfs(x, y, model_xgb), y))
print('ADB - ', adb(sfs(x, y, model_adb), y))
print('MLP - ', mlp(sfs(x, y, model_mlp), y))
print('GB - ', gb(sfs(x, y, model_gb), y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

AttributeError: SequentialFeatureSelector has not been fitted, yet.

In [40]:
#with feature selection rfe


print('lr - ', lr(rfe(x, y, model_lr), y))
print('dtc - ', dtc(rfe(x, y, model_dtc), y))
print('rfc - ', rfc(rfe(x, y, model_rfc), y))
print('knn - ', knn(rfe(x, y, model_knn), y))
# print('svm - ', svm(rfe(x, y, model_svc), y))
print('Naive Bayes - ', nb(rfe(x, y, model_nb), y))
print('XGB - ', xgbb(rfe(x, y, model_xgb), y))
print('ADB - ', adb(rfe(x, y, model_adb), y))
print('MLP - ', mlp(rfe(x, y, model_mlp), y))
print('GB - ', gb(rfe(x, y, model_gb), y))

C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\serev\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

KeyboardInterrupt: 

In [ ]:
#with feature selection csf


print('lr - ', lr(data_cfs, y))
print('dtc - ', dtc(data_cfs, y))
print('rfc - ', rfc(data_cfs, y))
print('knn - ', knn(data_cfs, y))
# print('svm - ', svm(data_cfs, y))
print('Naive Bayes - ', nb(data_cfs, y))
print('XGB - ', xgbb(data_cfs, y))
print('ADB - ', adb(data_cfs, y))
print('MLP - ', mlp(data_cfs, y))
print('GB - ', gb(data_cfs, y))

In [ ]:
#with feature selection fir


print('lr - ', lr(data_fir, y))
print('dtc - ', dtc(data_fir, y))
print('rfc - ', rfc(data_fir, y))
print('knn - ', knn(data_fir, y))
# print('svm - ', svm(data_fir, y))
print('Naive Bayes - ', nb(data_fir, y))
print('XGB - ', xgbb(data_fir, y))
print('ADB - ', adb(data_fir, y))
print('MLP - ', mlp(data_fir, y))
print('GB - ', gb(data_fir, y))